Perhaps, one of the simplest and most basic operations in Google Earth Engine is to retrieve information for a single point. Typical examples include elevation, mean annual temperature, and soil properties. In other words, this type of operations is mostly intended at retrieving information that does not change with time. 

In [1]:
# Import modules
import ee
from datetime import datetime
from pprint import pprint

In [2]:
# Trigger the authentication flow.
ee.Authenticate(auth_mode='localhost')

# Initialize the library.
ee.Initialize(project='ee-ializarazos')


Successfully saved authorization token.


## Example 1: Elevation

In the following example we will retrieve the elevation for a specific point on Earth.

**Product**: USGS/SRTMGL1_003

In [3]:
# Define geographic coordinates
# Bucaramanga
# 7.1193° N, 73.1227° W
lat = 7.1193 # This is y
lon = -73.1227 # This is x

# Convert coordinates into a Point geometry following the x,y notation
point = ee.Geometry.Point([lon, lat])

# Explore point geometry
point.getInfo()


{'type': 'Point', 'coordinates': [-73.1227, 7.1193]}

In [4]:
# Load digital elevation model (DEM) from Shuttle Radar Topography Mission (SRTM) 
dem = ee.Image('USGS/SRTMGL1_003')


In [5]:
# Obtain some information about the DEM layer (output is long!)
pprint(dem.getInfo())


{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [0.0002777777777777778,
                              0,
                              -180.0001388888889,
                              0,
                              -0.0002777777777777778,
                              60.00013888888889],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [1296001, 417601],
            'id': 'elevation'}],
 'id': 'USGS/SRTMGL1_003',
 'properties': {'date_range': [950227200000, 951177600000],
                'description': '<p>The Shuttle Radar Topography Mission (SRTM, '
                               'see <a '
                               'href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\n'
                               'et al. 2007</a>)\n'
                               'digital elevation data is an internation

In [6]:
# Retrieve the elevation value: This step will get us close to the answer, but we are not there yet
pprint(dem.sample(point, 1).getInfo())


{'columns': {'elevation': 'Short'},
 'features': [{'geometry': None,
               'id': '0',
               'properties': {'elevation': 967},
               'type': 'Feature'}],
 'properties': {'band_order': ['elevation']},
 'type': 'FeatureCollection'}


In [7]:
# Retrieve the elevation value: This step will get us even closer to the answer, but we are not there yet
dem.sample(point, 1).first().getInfo()


{'type': 'Feature',
 'geometry': None,
 'id': '0',
 'properties': {'elevation': 967}}

In [8]:
# Retrieve the elevation value: This step will get us the correct value
elev = dem.sample(point, 1).first().getNumber('elevation').getInfo()
print(f'Elevation: {elev} m')


Elevation: 967 m


## Example 2: Weather variables

Obtain the long-term mean annual air temperature and precipitation for a specific location

**Product**: For more information about bands and units visit: [WorldClim BIO](https://developers.google.com/earth-engine/datasets/catalog/WORLDCLIM_V1_BIO)

In [9]:
# Load WorldClim BIO dataset
dataset = ee.Image('WORLDCLIM/V1/BIO')


In [10]:
# Access metadata of the product (output is long!)
dataset.getInfo()


{'type': 'Image',
 'bands': [{'id': 'bio01',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [43200, 18000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333333333333,
    0,
    -180,
    0,
    -0.008333333333333333,
    90]},
  {'id': 'bio02',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [43200, 18000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333333333333,
    0,
    -180,
    0,
    -0.008333333333333333,
    90]},
  {'id': 'bio03',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [43200, 18000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333333333333,
    0,
    -180,
    0,
    -0.008333333333333333,
    90]},
  {'id': 'bio04',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [43200, 18000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333333333333,
    0,
    -180,
    0,
    -0.008333333333333333,
    90]},
  {'id': 'bio05',

In [11]:
# Get long-term mean annual air temperature
T_mean = dataset.select('bio01').sample(point,1).first().getNumber('bio01').multiply(0.1).getInfo()
print(f'Mean annual air temperature: {round(T_mean,1)} Celsius')


Mean annual air temperature: 23.3 Celsius


In [12]:
# Get long-term annual precipitation
P_annual = dataset.select('bio12').sample(point,1).first().get('bio12').getInfo()
print(f'Mean annual precipitation: {round(P_annual,1)} mm')


Mean annual precipitation: 1176 mm


## Example 3: Reference ET

In this example we will retrieve daily values of reference evapotranspiration for a point.

**Product**: For more information visit the description in Google Earth Engine of [GRIDMET](https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET?hl=en)


In [13]:
# Obtain GRIDMET dataset for a specific period. End date is excluded from the call
start_date = ee.Date('2022-12-01')
end_date = ee.Date('2023-01-01')
day_of_interest = ee.Date('2023-12-15')
dataset = ee.ImageCollection("FAO/WAPOR/2/L1_RET_E").filterDate(start_date, end_date)

In [14]:
# Information about image collection
pprint(dataset.getInfo())

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:4326',
                          'crs_transform': [0.178571,
                                            0,
                                            -179.8255333,
                                            0,
                                            -0.178571,
                                            60.0044163],
                          'data_type': {'max': 255,
                                        'min': 0,
                                        'precision': 'int',
                                        'type': 'PixelType'},
                          'dimensions': [2015, 673],
                          'id': 'L1_RET_E'}],
               'id': 'FAO/WAPOR/2/L1_RET_E/L1_RET_221201',
               'properties': {'system:asset_size': 79472,
                              'system:footprint': {'coordinates': [[-180, -90],
                                                                   [180, -90],
                       

In [15]:
# Use the get methog to retrieve a specific property
dataset.get('description').getInfo()


'<p>Reference evapotranspiration (RET) is defined as the evapotranspiration from a\nhypothetical reference crop and it simulates the behaviour of a well-watered\ngrass surface. Each pixel represents the daily reference evapotranspiration in mm.</p><p><b>Provider: <a href="https://wapor.apps.fao.org/catalog/WAPOR_2/1/L1_RET_E">FAO UN</a></b><br><p><b>Resolution</b><br>18924 meters\n</p><p><b>Cadence</b><br>\n  1 day\n</p><p><b>Bands</b><table class="eecat"><tr><th scope="col">Name</th><th scope="col">Description</th></tr><tr><td>L1_RET_E</td><td><p>Reference Evapotranspiration (Daily) [mm]</p></td></tr></table><p><b>Terms of Use</b><br><p>The Food and Agriculture Organization of the United Nations (FAO) is mandated to\ncollect, analyze, interpret, and disseminate information related to nutrition,\nfood, and agriculture. In this regard, it publishes a number of databases on topics\nrelated to FAO&#39;s mandate, and encourages the use of them for scientific and research purposes.\nConsist

In [16]:
# Information about first image within the collection (output is long!)
dataset.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'L1_RET_E',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [2015, 673],
   'crs': 'EPSG:4326',
   'crs_transform': [0.178571, 0, -179.8255333, 0, -0.178571, 60.0044163]}],
 'version': 1677333077102453,
 'id': 'FAO/WAPOR/2/L1_RET_E/L1_RET_221201',
 'properties': {'system:time_start': 1669852800000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1669852800000,
  'system:asset_size': 79472,
  'system:index': 'L1_RET_221201'}}

In [45]:
# Information about feature collection
point = ee.Geometry.Point([-76.013, 4.736])
dataset.first().sample(point,1).getInfo() # Sample at 1 meter resolution

{'type': 'FeatureCollection',
 'columns': {'L1_RET_E': 'Short<0, 255>'},
 'properties': {'band_order': ['L1_RET_E']},
 'features': []}

In [47]:
# Information about feature
#eto = dataset.first().sample(point,1).first().getNumber('L1_RET_E').getInfo()
print(f'The grass reference ET is {eto} mm')

EEException: Element.getNumber: Parameter 'object' is required and may not be null.

Alternative solution: Access the image directly rather than the collection.

```python
ee.Image('IDAHO_EPSCOR/GRIDMET/20220701').sample(point,1).first().getNumber('eto').getInfo()
```

#### Get dataset timestamps

In [37]:
# Obtain the START time of the dataset to check that it matches our request.
# Response is in milliseconds since 1-Jan-1970
dataset.select('eto').first().getNumber('system:time_start').getInfo()


1669852800000

In [38]:
# Obtain the END time of the dataset to check that it matches our request.
# Response is in milliseconds since 1-Jan-1970
dataset.select('eto').first().getNumber('system:time_end').getInfo()


1669852800000

In [39]:
# Find the datetime of the serial date numbers
# Input in "fromtimestamp()" has to be in seconds, so we divide by 1000
print('Start time:', datetime.fromtimestamp(1656655200000/1000).strftime('%Y-%m-%d %H:%M:%S.%f'))
print('End time:', datetime.fromtimestamp(1656741600000/1000).strftime('%Y-%m-%d %H:%M:%S.%f'))


Start time: 2022-06-30 23:00:00.000000
End time: 2022-07-01 23:00:00.000000


## Example 4: Retrieve soil properties for a given location

Product: SoilGrids

Source: https://samapriya.github.io/awesome-gee-community-datasets/projects/isric/

In [40]:
# Load the SoilGrids dataset from GEE
soil_grids = ee.Image("projects/soilgrids-isric/sand_mean")

In [43]:
# Define a point geometry
point = ee.Geometry.Point([-76.013, 4.736])
sand = soil_grids.sample(point,250).first().getNumber('sand_0-5cm_mean').multiply(0.1).getInfo()

In [44]:
print(f'The percentage of sand at ({lat},{lon}) is: {round(sand)} %')

The percentage of sand at (7.1193,-73.1227) is: 29 %
